# Download ELEPHANT server

In [ ]:
%env ELEPHANT_SERVER_VERSION=0.1.0

In [ ]:
!wget -q -c -nc https://github.com/elephant-track/elephant-server/releases/download/v$ELEPHANT_SERVER_VERSION/elephant-server-$ELEPHANT_SERVER_VERSION.zip
!unzip -qq -n elephant-server-$ELEPHANT_SERVER_VERSION.zip
!rm elephant-server-$ELEPHANT_SERVER_VERSION.zip

# Set up conda environment

In [ ]:
!printf "\
name: base \n\
channels: \n\
  - pytorch \n\
  - conda-forge \n\
  - main \n\
  - defaults \n\
dependencies: \n\
  - _libgcc_mutex=0.1=main \n\
  - asciitree=0.3.3=py_2 \n\
  - asn1crypto=1.2.0=py37_0 \n\
  - backcall=0.1.0=py37_0 \n\
  - beautifulsoup4=4.8.2=py37_0 \n\
  - blas=1.0=mkl \n\
  - bzip2=1.0.8=h7b6447c_0 \n\
  - ca-certificates=2021.1.19=h06a4308_0 \n\
  - certifi=2020.12.5=py37h06a4308_0 \n\
  - cffi=1.13.0=py37h2e261b9_0 \n\
  - chardet=3.0.4=py37_1003 \n\
  - click=7.1.2=pyhd3eb1b0_0 \n\
  - cloudpickle=1.6.0=py_0 \n\
  - conda=4.9.2=py37h06a4308_0 \n\
  - conda-build=3.18.11=py37_0 \n\
  - conda-package-handling=1.6.0=py37h7b6447c_0 \n\
  - cryptography=2.8=py37h1ba5d50_0 \n\
  - cudatoolkit=10.1.243=h6bb024c_0 \n\
  - cycler=0.10.0=py37_0 \n\
  - cytoolz=0.11.0=py37h7b6447c_0 \n\
  - dask-core=2021.2.0=pyhd3eb1b0_0 \n\
  - decorator=4.4.1=py_0 \n\
  - fasteners=0.16=pyhd3eb1b0_0 \n\
  - filelock=3.0.12=py_0 \n\
  - flask=1.1.2=pyhd3eb1b0_0 \n\
  - flask-redis=0.4.0=py_0 \n\
  - freetype=2.9.1=h8a8886c_1 \n\
  - glob2=0.7=py_0 \n\
  - h5py=2.10.0=py37hd6299e0_1 \n\
  - hdf5=1.10.6=hb1b8bf9_0 \n\
  - icu=58.2=he6710b0_3 \n\
  - idna=2.8=py37_0 \n\
  - imageio=2.9.0=py_0 \n\
  - intel-openmp=2019.4=243 \n\
  - ipython=7.11.1=py37h39e3cac_0 \n\
  - ipython_genutils=0.2.0=py37_0 \n\
  - itsdangerous=1.1.0=py37_0 \n\
  - jansson=2.11=h516909a_1001 \n\
  - jedi=0.15.2=py37_0 \n\
  - jinja2=2.10.3=py_0 \n\
  - joblib=1.0.1=pyhd3eb1b0_0 \n\
  - jpeg=9b=h024ee3a_2 \n\
  - kiwisolver=1.3.1=py37h2531618_0 \n\
  - libarchive=3.3.3=h5d8350f_5 \n\
  - libedit=3.1.20181209=hc058e9b_0 \n\
  - libffi=3.2.1=hd88cf55_4 \n\
  - libgcc-ng=9.1.0=hdf63c60_0 \n\
  - libgfortran-ng=7.3.0=hdf63c60_0 \n\
  - libiconv=1.15=h63c8f33_5 \n\
  - liblief=0.9.0=h7725739_2 \n\
  - libpng=1.6.37=hbc83047_0 \n\
  - libprotobuf=3.14.0=h8c45485_0 \n\
  - libstdcxx-ng=9.1.0=hdf63c60_0 \n\
  - libtiff=4.1.0=h2733197_0 \n\
  - libxml2=2.9.9=hea5a465_1 \n\
  - lz4-c=1.8.1.2=h14c3975_0 \n\
  - lzo=2.10=h49e0be7_2 \n\
  - markupsafe=1.1.1=py37h7b6447c_0 \n\
  - matplotlib-base=3.2.1=py37hef1b27d_0 \n\
  - mkl=2019.4=243 \n\
  - mkl-service=2.3.0=py37he904b0f_0 \n\
  - mkl_fft=1.0.15=py37ha843d7b_0 \n\
  - mkl_random=1.1.0=py37hd6b4f25_0 \n\
  - monotonic=1.5=py_0 \n\
  - msgpack-python=1.0.2=py37hff7bd54_1 \n\
  - ncurses=6.1=he6710b0_1 \n\
  - networkx=2.5=py_0 \n\
  - ninja=1.9.0=py37hfd86e86_0 \n\
  - numcodecs=0.7.3=py37h2531618_0 \n\
  - numpy=1.17.4=py37hc1035e2_0 \n\
  - numpy-base=1.17.4=py37hde5b4d6_0 \n\
  - olefile=0.46=py37_0 \n\
  - openssl=1.1.1j=h27cfd23_0 \n\
  - parso=0.5.2=py_0 \n\
  - patchelf=0.10=he6710b0_0 \n\
  - pcre=8.44=he6710b0_0 \n\
  - pexpect=4.7.0=py37_0 \n\
  - pickleshare=0.7.5=py37_0 \n\
  - pika=1.1.0=pyh9f0ad1d_1 \n\
  - pillow=7.0.0=py37hb39fc2d_0 \n\
  - pip=19.3.1=py37_0 \n\
  - pkginfo=1.5.0.1=py37_0 \n\
  - prompt_toolkit=3.0.2=py_0 \n\
  - protobuf=3.14.0=py37h2531618_1 \n\
  - psutil=5.6.7=py37h7b6447c_0 \n\
  - ptyprocess=0.6.0=py37_0 \n\
  - py-lief=0.9.0=py37h7725739_2 \n\
  - pycosat=0.6.3=py37h14c3975_0 \n\
  - pycparser=2.19=py37_0 \n\
  - pygments=2.5.2=py_0 \n\
  - pyopenssl=19.0.0=py37_0 \n\
  - pyparsing=2.4.7=pyhd3eb1b0_0 \n\
  - pysocks=1.7.1=py37_0 \n\
  - python=3.7.4=h265db76_1 \n\
  - python-dateutil=2.8.1=pyhd3eb1b0_0 \n\
  - python-libarchive-c=2.8=py37_13 \n\
  - pytorch=1.4.0=py3.7_cuda10.1.243_cudnn7.6.3_0 \n\
  - pytz=2019.3=py_0 \n\
  - pywavelets=1.1.1=py37h7b6447c_2 \n\
  - pyyaml=5.2=py37h7b6447c_0 \n\
  - readline=7.0=h7b6447c_5 \n\
  - redis-py=3.5.3=py_0 \n\
  - requests=2.22.0=py37_0 \n\
  - ripgrep=11.0.2=he32d670_0 \n\
  - ruamel_yaml=0.15.46=py37h14c3975_0 \n\
  - scikit-image=0.17.2=py37hdf5156a_0 \n\
  - scikit-learn=0.23.1=py37h423224d_0 \n\
  - scipy=1.4.1=py37h0b6359f_0 \n\
  - setuptools=41.4.0=py37_0 \n\
  - six=1.12.0=py37_0 \n\
  - soupsieve=1.9.5=py37_0 \n\
  - sqlite=3.30.0=h7b6447c_0 \n\
  - tensorboardx=2.1=py_0 \n\
  - threadpoolctl=2.1.0=pyh5ca1d4c_0 \n\
  - tifffile=2020.10.1=py37hdd07704_2 \n\
  - tk=8.6.8=hbc83047_0 \n\
  - toolz=0.11.1=pyhd3eb1b0_0 \n\
  - torchvision=0.5.0=py37_cu101 \n\
  - tornado=6.1=py37h27cfd23_0 \n\
  - tqdm=4.48.2=py_0 \n\
  - traitlets=4.3.3=py37_0 \n\
  - urllib3=1.24.2=py37_0 \n\
  - uwsgi=2.0.18=py37hd004684_2 \n\
  - wcwidth=0.1.7=py37_0 \n\
  - werkzeug=1.0.1=pyhd3eb1b0_0 \n\
  - wheel=0.33.6=py37_0 \n\
  - xz=5.2.4=h14c3975_4 \n\
  - yaml=0.1.7=had09818_2 \n\
  - zarr=2.4.0=py_0 \n\
  - zlib=1.2.11=h7b6447c_3 \n\
  - zstd=1.3.7=h0b5b093_0 \n\
  - pip: \n\
    - ./elephant-server-0.1.0/elephant-core  \
" > environment.yml

In [ ]:
# Acknowledgements:
#     https://donaldsrepo.github.io/Notebooks/GoogleColabCondaCreateEnv.html
conda_path = ''
try:
    conda_path = !which conda
finally:
    print('')

if (len(conda_path) == 0):
    print('installing miniconda')
    !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    !bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
    !rm Miniconda3-4.5.4-Linux-x86_64.sh
    !conda update conda -y -q
    !source /usr/local/etc/profile.d/conda.sh
    !conda init    
else:
    print('found miniconda')

!conda env update -f environment.yml

# Prepare Demo dataset

In [ ]:
# Set up dirs
!mkdir -p /workspace/models
!mkdir -p /workspace/datasets/elephant-demo
# Download files
!curl -L https://zenodo.org/record/4549193/files/elephant-demo.h5?download=1 \
  -o /workspace/datasets/elephant-demo/elephant-demo.h5
!curl -L https://zenodo.org/record/4549193/files/elephant-demo.xml?download=1 \
  -o /workspace/datasets/elephant-demo/elephant-demo.xml
# Run script
!python /content/elephant-server-$ELEPHANT_SERVER_VERSION/script/dataset_generator.py \
  --uint16 /workspace/datasets/elephant-demo/elephant-demo.h5 /workspace/datasets/elephant-demo

# Install apt packages

In [ ]:
!apt-get update && apt-get install --no-install-recommends --no-install-suggests -y \
    nginx \
    redis-server \
    supervisor \
    ca-certificates \
    curl \
    gnupg \
    gosu \
    openssh-server \
    pwgen

# Install RabbitMQ

In [ ]:
!curl -fsSL https://github.com/rabbitmq/signing-keys/releases/download/2.0/rabbitmq-release-signing-key.asc | apt-key add - && \
    apt-get update \
    && apt-get install --no-install-recommends --no-install-suggests -y \
    apt-transport-https && \
    echo 'deb http://dl.bintray.com/rabbitmq-erlang/debian bionic erlang-22.x\ndeb https://dl.bintray.com/rabbitmq/debian bionic main' >> /etc/apt/sources.list.d/bintray.rabbitmq.list && \
    echo 'Package: erlang*\nPin: release o=Bintray\nPin-Priority: 1000' >> /etc/apt/preferences.d/erlang && \
    apt-get update \
    && apt-get install --no-install-recommends --no-install-suggests -y \
    rabbitmq-server

# Set up services

In [ ]:
# Set up RabbitMQ
%env RABBITMQ_USER=user
%env RABBITMQ_PASSWORD=user
%env RABBITMQ_PID_FILE=/var/lib/rabbitmq/mnesia/rabbitmq.pid
!cp /content/elephant-server-$ELEPHANT_SERVER_VERSION/docker/rabbitmq.sh /rabbitmq.sh
!sed -i 's;--timeout 60 ;;g' /rabbitmq.sh
!chmod +x /rabbitmq.sh
# Set up nginx
!cp /content/elephant-server-$ELEPHANT_SERVER_VERSION/docker/nginx.conf /etc/nginx/nginx.conf
!groupadd nginx && useradd -g nginx nginx
!ln -sf /dev/stdout /var/log/nginx/access.log
!ln -sf /dev/stderr /var/log/nginx/error.log
# Set up uWSGI
!mkdir -p /etc/uwsgi
!cp /content/elephant-server-$ELEPHANT_SERVER_VERSION/docker/uwsgi.ini /etc/uwsgi/uwsgi.ini
!cp -a /content/elephant-server-$ELEPHANT_SERVER_VERSION/app /app
!useradd -u 0 -o -m user
%env UWSGI_UID=user
%env UWSGI_GID=user
# Set up supervisor
!cp /content/elephant-server-$ELEPHANT_SERVER_VERSION/docker/supervisord.conf /etc/supervisor/conf.d/supervisord.conf
!sed -i 's;/opt/conda/bin/uwsgi;uwsgi;g' /etc/supervisor/conf.d/supervisord.conf
# Set up sshd
!mkdir -p /var/run/sshd
!grep -qxF "PermitRootLogin yes" /etc/ssh/sshd_config || echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!grep -qxF "PasswordAuthentication yes" /etc/ssh/sshd_config || echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

# Download ngrok

In [ ]:
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip -d /opt/ngrok
!rm ngrok-stable-linux-amd64.zip

# Run sshd with ngrok and start services

In [ ]:
# Acknowledgements:
# @Tamlyn https://stackoverflow.com/a/53252985
import getpass
import json
import random
import string
import urllib.request
import time

#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Set root password
!echo root:$password | chpasswd

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('/opt/ngrok/ngrok authtoken $authtoken && /opt/ngrok/ngrok tcp 22 &')

#Get public address and print connect command
time.sleep(1)
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  for tunnel in data['tunnels']:
    if tunnel['name'] == 'command_line':
      (host, port) = tunnel['public_url'][6:].split(':')
      print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')

# Run services
%cd /app
!/usr/bin/supervisord